# CAMS — Algorithmic Project: Math & Logic Patterns
**Author:** Swayam Gharat  
**Date:** November 13, 2025  

---

This notebook explores fundamental algorithmic patterns and mathematical logic useful for CAMS projects and coding interviews.  
It contains explanations, implementations, visualisations, and brief complexity discussions. Run cells sequentially.

**Sections**
1. Setup & verification
2. Prime number analysis
3. Fibonacci: iterative vs recursive
4. Sorting logic: Bubble Sort & Merge Sort (counts & timings)
5. Pattern generation: Pascal's triangle and numeric triangles
6. Mathematical insights & conclusions


## 1) Setup & verification
This cell imports required libraries and verifies the runtime environment.

In [ ]:
# Setup & verification
import sys, time, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print('Python version:', sys.version.split()[0])
print('NumPy version:', np.__version__)
print('Pandas version:', pd.__version__)
print('Matplotlib version:', plt.__version__)

# simple check
print('\nQuick check: array operations ->', np.array([1,2,3]) * 2)

## 2) Prime Number Analysis
- We'll implement an efficient sieve (Sieve of Eratosthenes) to generate primes up to `n`.
- We'll visualise prime density (primes count vs n).

In [ ]:
def sieve_primes(n):
    """Return list of primes <= n using Sieve of Eratosthenes."""
    if n < 2:
        return []
    sieve = bytearray(b'\x01') * (n+1)
    sieve[0:2] = b'\x00\x00'
    for p in range(2, int(n**0.5) + 1):
        if sieve[p]:
            step = p
            start = p*p
            sieve[start:n+1:step] = b'\x00' * (((n - start)//step) + 1)
    return [i for i, isprime in enumerate(sieve) if isprime]

# generate primes and plot density
n = 5000
primes = sieve_primes(n)
counts = [len(sieve_primes(i)) for i in range(1, n+1, 50)]  # sample counts
xs = list(range(1, n+1, 50))

print('Number of primes <=', n, ':', len(primes))
plt.figure(figsize=(8,4))
plt.plot(xs, counts)
plt.title('Prime count vs n (sampled every 50)')
plt.xlabel('n')
plt.ylabel('count of primes ≤ n')
plt.grid(True)
plt.show()

**Complexity note:** Sieve runs in \(O(n \log \log n)\) time and uses \(O(n)\) space. Prime density roughly follows \(\pi(n) \sim n / \ln n\).

## 3) Fibonacci — Iterative vs Recursive
We'll implement both versions, compare outputs and timing. Recursive naive Fibonacci has exponential time and is used to illustrate why recursion without memoization is slow.

In [ ]:
from functools import lru_cache
def fib_iter(n):
    if n <= 1:
        return n
    a, b = 0, 1
    for _ in range(2, n+1):
        a, b = b, a + b
    return b if n>0 else a

@lru_cache(maxsize=None)
def fib_rec_memo(n):
    if n < 2:
        return n
    return fib_rec_memo(n-1) + fib_rec_memo(n-2)

def fib_rec_naive(n):
    if n < 2:
        return n
    return fib_rec_naive(n-1) + fib_rec_naive(n-2)

# Compare timings
N = 30
t0 = time.time(); r_iter = fib_iter(N); t1 = time.time()
t_iter = t1 - t0

t0 = time.time(); r_memo = fib_rec_memo(N); t1 = time.time()
t_memo = t1 - t0

t0 = time.time(); r_naive = fib_rec_naive(20); t1 = time.time()
t_naive = t1 - t0

print(f'fib({N}) iterative =', r_iter, f' (time {t_iter:.6f}s)')
print(f'fib({N}) recursive+memo =', r_memo, f' (time {t_memo:.6f}s)')
print('fib(20) naive recursive =', r_naive, f' (time {t_naive:.6f}s)\n')

# Plot timings for naive recursion vs iterative for growing n (limited range for naive)
ns = list(range(5, 31))
times_iter = []
times_memo = []
times_naive = []

for n in ns:
    t0=time.time(); fib_iter(n); times_iter.append(time.time()-t0)
    t0=time.time(); fib_rec_memo(n); times_memo.append(time.time()-t0)
    if n <= 22: # avoid very long runs for naive
        t0=time.time(); fib_rec_naive(n); times_naive.append(time.time()-t0)
    else:
        times_naive.append(None)

plt.figure(figsize=(8,4))
plt.plot(ns, times_iter, label='iterative')
plt.plot(ns, times_memo, label='recursive+memo')
plt.plot(ns, times_naive, label='naive recursive (up to n=22)')
plt.yscale('log')
plt.xlabel('n'); plt.ylabel('time (s) [log scale]')
plt.legend(); plt.grid(True)
plt.title('Fibonacci: time complexity comparison')
plt.show()

**Insight:** Use iteration or memoization to avoid exponential time. Iterative is memory-light; memoized recursion is simple and fast for many problems.

## 4) Sorting Logic & Complexity
We'll implement Bubble Sort (O(n^2)) and Merge Sort (O(n log n)), count comparisons, and compare performance on random arrays.

In [ ]:
import random

def bubble_sort_count(arr):
    a = arr.copy()
    n = len(a)
    comps = 0
    for i in range(n):
        for j in range(0, n-i-1):
            comps += 1
            if a[j] > a[j+1]:
                a[j], a[j+1] = a[j+1], a[j]
    return a, comps

def merge_sort_count(arr):
    comps = 0
    def merge(left, right):
        nonlocal comps
        i=j=0
        out=[]
        while i < len(left) and j < len(right):
            comps += 1
            if left[i] <= right[j]:
                out.append(left[i]); i+=1
            else:
                out.append(right[j]); j+=1
        out.extend(left[i:]); out.extend(right[j:])
        return out
    def ms(a):
        if len(a) <= 1:
            return a
        mid = len(a)//2
        return merge(ms(a[:mid]), ms(a[mid:]))
    res = ms(arr)
    return res, comps

# Test and compare on random arrays
sizes = [50, 200, 800]
results = []
for s in sizes:
    arr = [random.randint(0, 10000) for _ in range(s)]
    t0=time.time(); _, c1 = bubble_sort_count(arr); t1=time.time(); tb = t1-t0
    t0=time.time(); _, c2 = merge_sort_count(arr); t1=time.time(); tm = t1-t0
    results.append({'size': s, 'bubble_comps': c1, 'merge_comps': c2, 'bubble_time': tb, 'merge_time': tm})

df = pd.DataFrame(results)
df

**Observation:** Bubble sort comparisons grow ~n^2, merge sort behaves ~n log n. Use built-in `sorted()` for production but implement algorithms to understand behaviour.

## 5) Pattern Generation Algorithms
Examples: Pascal's triangle and a numeric triangle generator. These strengthen combinatorial and visual intuition.

In [ ]:
def pascal_triangle(rows):
    res = []
    for r in range(rows):
        row = [1]
        if r > 0:
            prev = res[-1]
            for i in range(1, r):
                row.append(prev[i-1] + prev[i])
            row.append(1)
        res.append(row)
    return res

def numeric_triangle(n):
    res = []
    num = 1
    for i in range(1, n+1):
        row = list(range(num, num+i))
        num += i
        res.append(row)
    return res

# Display small examples
pt = pascal_triangle(7)
nt = numeric_triangle(6)

print('Pascal triangle (7 rows):')
for row in pt:
    print(row)
print('\nNumeric triangle (6 rows):')
for row in nt:
    print(row)

You can visualise Pascal's triangle row sums (which are powers of 2) and use it to explain combinations (n choose k).

## 6) Mathematical Insights & Conclusions
- Implementations show why algorithmic choices matter.
- For large inputs prefer algorithms with better asymptotic complexity.
- This notebook can be extended with experiments (e.g., empirical runtimes across more sizes) for your CAMS report.

---

**Submit this notebook as part of your project.** You can copy markdown cells into a report and include plots as figures.

---

*Good luck, Swayam — let me know if you want a PDF export or a cleaned 'report' version of this notebook.*